<a href="https://colab.research.google.com/github/MarcoGiordano96/DS_works/blob/master/GSC_Analysis_%5BQuery_Count%2C_N_grams_and_Clustering%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyzing data with Google Search Console to find opportunities and assess query count

Google Search Console (GSC) is a free tool familiar to many SEO Specialists. However, most of them don't use it to its full potential, which can be exploited via the official API.

A common SEO task involves checking GSC to spot opportunities for new queries or to assess your Organic Performance. Using Excel and/or spreadsheets is time consuming and can get in your way if you find to use more advanced functions. Python can solve all these issues and give you much more, as I will show you. You can also use this Google Colab file as a basis for reporting, just change something and comment the due parts and then download it from "File > Download".

If you want to copy this notebook, go to "File > Save a copy in Drive" and you are all set to start.

We will mainly cover basic data wrangling and analysis, plus some NLP concepts such as n-grams and association rule mining, a set of convenient techniques to investigate data. But don't worry, you don't need to understand it all now, just be sure to follow what's written and the business purpose/impact of what we have found.

This notebook is currently at its first version, but many more updates are planned in the next future. Please, follow the instructions and enjoy the newfound insights!

N.B. This notebook took inspiration in some parts from [Hamlet Baptista's notebook](https://colab.research.google.com/github/hamletbatista/sej/blob/master/Hands_on_Introduction_to_Machine_Learning_for_SEOs.ipynb) and [this one from RankSense](https://github.com/ranksense/Twittorials/blob/master/Analyzing_GSC_Data.ipynb).


Before we are able to import data from Google Search Console, there is some necessary setup to be made:

1. Having acces to Google Cloud Platform and enabling the **Google Search Console API** from [this link](https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId=). 
2. Create new credentials of type OAuth 2.0 in order to request authentication by following [this link](https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project=).
3. Download a copy of the id in .json format, that will be uploaded here in Google Colab.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
%%capture
#load what is needed
!pip install git+https://github.com/joshcarty/google-searchconsole
import pandas as pd
import numpy as np
import matplotlib as plt
from google.colab import data_table
!git clone https://github.com/jroakes/querycat.git
!pip install -r querycat/requirements_colab.txt
!pip install umap-learn
data_table.enable_dataframe_formatter() #for better table visualization


In [ ]:
import searchconsole
account = searchconsole.authenticate(client_config='client_secret_.json',serialize='credentials.json', flow='console')

In [ ]:
property_name = input('Insert the name of your website as listed in GSC: ')
webproperty=account[str(property_name)]

We can create a function to extract data from one of our properties in Google Search Console. You can select as many dimensions as you want within the fuction, my suggestion is to proceed with query, page and date as we will need them all.

Queries alone may not capture all the information we want and that is exactly why we want pages in our dataset too. Dates are useful to understand what happens in the selected timespan.

In [ ]:
def extract_gsc_data(webproperty, start, stop, *args):
  if webproperty is not None:
    print(f'Extracting data for {webproperty}')
    gsc_data = webproperty.query.range(start, stop).dimension(*args).get()
    return gsc_data
  else:
    print('Webproperty not found, please select the correct one')
    return None


In [ ]:
#example of querying GSC API to get data
ex = extract_gsc_data(webproperty, '2021-12-01', '2021-12-01', 'query', 'page', 'date')

In [ ]:
df = pd.DataFrame(data=ex)
df.head()

In [ ]:
#how many unique pages in the dataset?
pages = list(set(df.page))
len(pages)

Now we analyze the average CTR per position group; to do so we need to first round the position column.

In [ ]:
#round position to have accurate groups and then create a pivot table for position and ctr, using the mean to aggregate
df['position'] = df['position'].round(0).astype('int64')
query_analysis = df.pivot_table(index=['position'], values=['ctr'], aggfunc=['mean'])

In [ ]:
pd.options.display.float_format = '{:.2%}'.format
query_analysis.sort_values(by=['position'], ascending=True).head(10)

In [ ]:
ax = query_analysis.head(10).plot(kind='bar')
ax.set_xlabel('Position')
ax.set_ylabel('CTR')
ax.set_title('CTR by position')
ax.grid('on')
ax.get_legend().remove()

You can expect Position 1 to have a much higher share and that is quite normal. This plot is handy to get the general distribution and especially to assess any incongruences. For instance, if position 7 displays a much higher mean value than position 5 you may want to investigate.

## Query count and SEO efforts

As the name implies, query count refers to the process of counting queries and then grouping them by the rounded position. The main idea here is that you can check in which positions you have the most queries and where you have more room for improvement.

For instance, if you notice that you have a lot of queries in position 7, you may want to investigate further to see if there are some quick wins of if there isn't much to do.

In [ ]:
ranking_queries = df.pivot_table(index=['position'], values=['query'], aggfunc=['count'])
ranking_queries.sort_values(by=['position']).head(10)

In [ ]:
ax = ranking_queries.head(10).plot(kind='bar')
ax.set_ylabel('Count of queries')
ax.set_title('Ranking distribution')
ax.grid('on')
ax.get_legend().remove()

Query count is one of the most important actions you can perform with GSC data. Although one can argue that quality trumps over quantity, an increase in query count can be related to SEO success and it is an indicator that Google is valuing more your website.

In an ideal scenario, the plot would be shifted leftward as you want to be very high under normal circumstances.

## The importance of non-branded queries

When measuring your organic performance you should always filter out branded keywords. Nonetheless, it is still of use to know how many branded queries you have and what people are looking for.

This section is aimed at understanding the relationship between branded and non-branded keywords.

In [ ]:
domain_name = str(input('Insert brand terms separated by a comma: ')).replace(',', '|')
import re
domain_name = re.sub(r"\s+", "", domain_name)
print('Remove all spaces using RegEx:\n')
df['Brand/Non-branded'] = np.where(
    df['query'].str.contains(domain_name), 'Brand', 'Non-branded'
)

In [ ]:
brand_count_df = df['Brand/Non-branded'].value_counts().rename_axis('cats').to_frame('counts')

In [ ]:
brand_count_df['Percentage'] = brand_count_df['counts']/sum(brand_count_df['counts'])

In [ ]:
pd.options.display.float_format = '{:.2%}'.format
brand_count_df

In [ ]:
df['Brand/Non-branded'].value_counts().plot(kind='bar')

Before we move on, it is very suggested to filter out branded keywords as they won't be part of our next analyses.

In [ ]:
#only select non-branded keywords
df = df.loc[df['Brand/Non-branded'] == 'Non-branded']

## NLP: how to leverage basic functions


Natural Language Processing (NLP) is one of the most interesting subsets of Machine Learning that are of interest for any SEO practitioner. Python is quite convenient for NLP tasks as there are plenty of packages and modules available.

One of the first tasks that comes nifty with this type of text data is n-gram analysis. In short, we want to know which are the most common sequence of words across our queries.

In [ ]:
import nltk
textlist = df['query'].to_list()
pd.Series(textlist).value_counts().head(10)

In [ ]:
from collections import Counter
x = Counter(textlist)
x.most_common(10)

In [ ]:
#download stopwords list to remove what is not needed
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

This very useful snippet of code was borrowed from this [Medium article](https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5://), which a good introduction to text analysis in Python.

In [ ]:
#create dataframe with bigrams and trigrams
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3)) #can also select bigrams only
# matrix of ngrams
ngrams = c_vec.fit_transform(df['query'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

Analyzing bigrams and trigrams is useful to understand your most popular topics, in a way. Using other techniques to cluster groups of words is definitely more effective, although n-gram analysis is still very practical and useful to identify clusters.

In [ ]:
df_ngram.head(20).style.background_gradient()

If you want to be more granular, you can check those queries that are above the 80th quantile for impressions and below the 20th quantile for CTR. In other words, we are looking for those queries who have more impressions than the other 80% of the dataset but have way worse CTR compared to 80% of the dataset.

In [ ]:
top_impressions = df[df['impressions'] >= df['impressions'].quantile(0.8)]
(top_impressions[top_impressions['ctr'] <= top_impressions['ctr'].quantile(0.2)].sort_values('impressions', ascending = False))

## Querycat to the rescue
A more effective way to tackle clustering is offered by the querycat library, capable of classifying queries with the help of association rule learning. We have already loaded and installed all the needed packages before so we are now ready to go!

NB: I will update this section to include the FP Growth algorithm, which is generally faster than the one I am going to show you.

In [ ]:
import querycat

Explaining querycat is not as straightforward as it seems as you may need to know more about some algorithms like the apriori one. For the moment, you just have to know we are using the apriori algorith with a value of 10 for the minimum support, which is quite good in terms of results.

The apriori algorithm is needed for finding similarities across rows and create categories based on that. Therefore, this methodology is also very useful when handling products  and/or categories in an e-commerce scenario, although we will use queries in this specific case.

In [ ]:
query_cat = querycat.Categorize(df, 'query', min_support=10, alg='apriori')

You may happen to have a lot of clusters and some of them may be small. You can set a filter to remove them.

In [ ]:
#if you have too many clusters, you can try to set a condition to remove those with very low clicks and suchlike
dfgrouped = df.groupby('category').agg(sumclicks = ('clicks', 'sum')).sort_values('sumclicks', ascending=False)
filtergroup = dfgrouped[dfgrouped['sumclicks'] > 15] #arbitrary threshold
filtergroup

In [ ]:
#merge original dataframe with filtered one
df = df.merge(filtergroup, on=['category','category'], how='inner')

In [ ]:
df['category'].value_counts()

In [ ]:
df.groupby('category').sum()['clicks'].sort_values(ascending=False).head(10)

We are interested in getting queries for each cluster along with total clicks and total impressions. To do this, we can create two variables storing the sum of clicks and impressions per group and then merge them to the other dataset.

In [ ]:
df.to_csv('df_gsc.csv')
files.download('df_gsc.csv')

In [ ]:
grouped_df = df.groupby('category')[['clicks', 'impressions']].agg('sum')

In [ ]:
group_ex = df.groupby(['category'])['query'].apply(' | '.join).reset_index()
#remove duplicate queries and then sort them alphabetically
group_ex['query'] = group_ex['query'].apply(lambda x: ' | '.join(sorted(list(set(x.split('|'))))))

In [ ]:
#final dataset ready to be exported
df_final = group_ex.merge(grouped_df, on=['category', 'category'], how='inner')
df_final.head()

In [ ]:
#save csv file and download it to your local machine. If you use Safari, please consider switching to Chrome for downloading these files as it may not work.
df_final.to_csv('clusters_queries.csv')
files.download('clusters_queries.csv')

Now you have a list of queries grouped by query with clicks and impressions to guide you in the decision process. Groups with a high impression count may be worth investigating, as you can find new opportunities or see if you can optimize for some queries, starting from your topical map. Please recall that optimizing for single queries alone is not enough anymore, and you should instead focus on organized groups of content where everything is interconnected via significant internal links with proper anchor texts.

In fact, this notebook should guide you in finding possible content gaps or to check how your clusters are performing.

For more information about this notebook or if you want to collaborate or provide me with a feedback, you can contact me on [Twitter](https://twitter.com/giordmarco96/) or [LinkedIn](https://www.linkedin.com/in/marco-giordano96).